In [1]:
from logging import getLogger
from itertools import product
from utils.dataset import RecDataset
from utils.dataloader import TrainDataLoader, EvalDataLoader
from utils.logger import init_logger
from utils.configurator import Config
from utils.utils import init_seed, get_model, get_trainer, dict2str
import platform
import os

### Dataset
baby, clothing, elec 카테고리로 이루어져 있음. 

(ex)baby 
baby.inter - (UserID, itemID, rating, timestamp, x_label(???)) 
i_id_mapping - (asin, itemID) asin- item code (ex)097293751X	0
u_id_mapping - (asin, userOD) (ex)097293751X	0
image_feat.npy - item image feature embedding
text_feat.npy - text image feature embedding
user_graph_dict.npy - 



In [2]:
import numpy as np

# .npy 파일 경로
file_path = '/home/sh0408sh/MMRec/data/baby-20240717T113127Z-001/baby/image_feat.npy'

# .npy 파일 읽기
data = np.load(file_path, allow_pickle=True)

# 데이터 확인
print(data)
print("배열의 형태:", data.shape) # (7050, 4096) - 7050개의 item image feature embedding(dimension = 4096)
print("배열의 데이터 타입:", data.dtype)

[[0.         0.         0.         ... 0.         0.56620002 1.2457    ]
 [0.         0.         0.         ... 0.         1.88339996 0.23909999]
 [0.         0.         0.         ... 0.         0.         4.71360016]
 ...
 [0.         0.         0.         ... 1.86619997 0.         0.        ]
 [1.61899996 0.91079998 0.         ... 1.51030004 0.         2.83529997]
 [0.         0.         0.         ... 0.         0.42070001 4.59919977]]
배열의 형태: (7050, 4096)
배열의 데이터 타입: float64


num of item : 7050 
item image embedding : 4096
item text embedding : 384

### main.py
model, dataset, Mirror Gradient값을 받아서 quick start  실행

(ex)python main.py --model BM3 --dataset baby mg --True

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', '-m', type=str, default='SELFCFED_LGN', help='name of models')
    parser.add_argument('--dataset', '-d', type=str, default='baby', help='name of datasets')
    parser.add_argument('--mg', action="store_true", help='whether to use Mirror Gradient, default is False')

    config_dict = {
        'gpu_id': 0,
    }

    args, _ = parser.parse_known_args()

    quick_start(model=args.model, dataset=args.dataset, config_dict=config_dict, save_model=True, mg=args.mg)
    

### quick_start의 구조

input : model, dataset
1. merg config dict : 학습에 관한 정보 config 객체를 만듦
2. Dataset 객체 생성 : 
3. DataLoader
4. Run model

1. merge config dict

MMRec/src/utils/configurator.py

In [3]:
# merge config dict
config_dict = {
        'gpu_id': 0,
    }
model = 'BM3'
dataset = 'baby'


config = Config(model, dataset, config_dict, mg = False)
config

# _load_dataset_model_config 함수 :  config + overall.yaml + dataset.yaml + model.yaml
# overall - training setting /home/sh0408sh/MMRec/src/configs/overall.yaml
# dataset.yaml 
# model. yaml - hyper parameter에 대한 정보


gpu_id=0
use_gpu=True
seed=[999]
data_path=../data/
inter_splitting_label=x_label
filter_out_cod_start_users=True
is_multimodal_model=True
checkpoint_dir=saved
save_recommended_topk=True
recommend_topk=recommend_topk/
embedding_size=64
weight_decay=0.0
req_training=True
epochs=1000
stopping_step=20
train_batch_size=2048
learner=adam
learning_rate=0.001
learning_rate_scheduler=[1.0, 50]
eval_step=1
training_neg_sample_num=1
use_neg_sampling=False
use_full_sampling=False
NEG_PREFIX=neg__
USER_ID_FIELD=userID
ITEM_ID_FIELD=itemID
TIME_FIELD=timestamp
field_separator=	
metrics=['Recall', 'NDCG', 'Precision', 'MAP']
topk=[5, 10, 20, 50]
valid_metric=Recall@20
eval_batch_size=4096
use_raw_features=False
max_txt_len=32
max_img_size=256
vocab_size=30522
type_vocab_size=2
hidden_size=4
pad_token_id=0
max_position_embeddings=512
layer_norm_eps=1e-12
hidden_dropout_prob=0.1
end2end=False
hyper_parameters=['seed', 'n_layers', 'reg_weight', 'dropout']
inter_file_name=baby.inter
vision_feature_file

2. Dataset 객체 생성

MMRec/src/utils/dataset.py

In [6]:
# load data : config에 있는 정보들을 이용해 RecDataset에 대한 객체 생성. 객체 내의 attribute에 dataset에 대한 다양한 정보가 포함됨

dataset = RecDataset(config)
dataset

baby
The number of users: 19445
Average actions of users: 8.269066598097197
The number of items: 7050
Average actions of items: 22.807375886524824
The number of inters: 160792
The sparsity of the dataset: 99.8827082752043%

In [7]:
print(dataset.dataset_name)
print(dataset.dataset_path)

baby
/home/sh0408sh/MMRec/data/baby


In [8]:
print(dataset.uid_field)
print(dataset.iid_field)
print(dataset.splitting_label)

userID
itemID
x_label


In [9]:
dataset.df

userID  itemID  x_label
0            0    1587        0
1            0    1879        0
2            0       0        0
3            0    1922        1
4            0    3870        2
...        ...     ...      ...
160787   19444    7022        0
160788   19444    6959        0
160789   19444    7005        0
160790   19444    7023        1
160791   19444    6994        2

[160792 rows x 3 columns]

In [10]:
# load rating file from data path?
print(config['inter_file_name'])
print(dataset.load_inter_graph(config['inter_file_name']))
print(dataset.item_num)
print(dataset.user_num)

baby.inter
None
7050
19445


In [ ]:
# RecDataset의 load_inter_graph 함수
def load_inter_graph(self, file_name):
        inter_file = os.path.join(self.dataset_path, file_name)
        cols = [self.uid_field, self.iid_field, self.splitting_label]
        self.df = pd.read_csv(inter_file, usecols=cols, sep=self.config['field_separator'])
        if not self.df.columns.isin(cols).all():
            raise ValueError('File {} lost some required columns.'.format(inter_file))

In [11]:
# data split
train_dataset, valid_dataset, test_dataset = dataset.split()
# data spliting label (x label)을 이용해 data slit

In [ ]:
# split 함수 구조
def split(self):
        dfs = []
        # splitting into training/validation/test
        for i in range(3):
            temp_df = self.df[self.df[self.splitting_label] == i].copy()
            temp_df.drop(self.splitting_label, inplace=True, axis=1)        # no use again
            dfs.append(temp_df)
        if self.config['filter_out_cod_start_users']:
            # filtering out new users in val/test sets
            train_u = set(dfs[0][self.uid_field].values)
            for i in [1, 2]:
                dropped_inter = pd.Series(True, index=dfs[i].index)
                dropped_inter ^= dfs[i][self.uid_field].isin(train_u)
                dfs[i].drop(dfs[i].index[dropped_inter], inplace=True)

3. Dataloader : config(학습에 대한 정보)와 dataset 객체를 통합함. 그외에 batch size, negative sampling, shuffle 유무에 대한 정보도 더해줌

class TrainDataLoader(AbstractDataLoader), EvalDataLoader(AbstractDataLoader)


MMRec/src/utils/dataloader.py

In [12]:
train_dataset.inter_num 
# self.sparsity = 1 - self.dataset.inter_num / self.dataset.user_num / self.dataset.item_num
# data loader내의 sparsity에 대한 식의 dataset.iner_num이 error를 일으킴. 
# dataset.inter_num은 RecDataset class 의 __str__함수에서 생성되는 attribute임. 전역 변수가 아니라서?
# 근데 파일을 돌릴 때는 단 한 번도 오류가 난적이 없고 여기서 간단하게 돌려볼 때도 에러가 났다가 안났다함.

#일단은 self.dataset.inter_num을 __str__안의 정의 되로 수정해주었음
# self.sparsity = 1 - len(self.dataset.df)/ self.dataset.user_num / self.dataset.item_num

AttributeError: 'RecDataset' object has no attribute 'inter_num'

In [13]:
# from utils.dataloader import TrainDataLoader, EvalDataLoader
# wrap into dataloader
train_data = TrainDataLoader(config, train_dataset, batch_size=config['train_batch_size'], shuffle=True)

In [14]:
(valid_data, test_data) = (
    EvalDataLoader(config, valid_dataset, additional_dataset=train_dataset, batch_size=config['eval_batch_size']),
    EvalDataLoader(config, test_dataset, additional_dataset=train_dataset, batch_size=config['eval_batch_size']))

In [ ]:
class TrainDataLoader(AbstractDataLoader):
    """
    General dataloader with negative sampling.
    """
    def __init__(self, config, dataset, batch_size=1, shuffle=False):
        super().__init__(config, dataset, additional_dataset=None,
                         batch_size=batch_size, neg_sampling=True, shuffle=shuffle)

        # special for training dataloader
        self.history_items_per_u = dict()
        # full items in training.
        self.all_items = self.dataset.df[self.dataset.iid_field].unique().tolist()
        self.all_uids = self.dataset.df[self.dataset.uid_field].unique()
        self.all_items_set = set(self.all_items)
        self.all_users_set = set(self.all_uids)
        self.all_item_len = len(self.all_items)
        # if full sampling
        self.use_full_sampling = config['use_full_sampling']

        if config['use_neg_sampling']:
            if self.use_full_sampling:
                self.sample_func = self._get_full_uids_sample
            else:
                self.sample_func = self._get_neg_sample
        else:
            self.sample_func = self._get_non_neg_sample

        self._get_history_items_u()
        self.neighborhood_loss_required = config['use_neighborhood_loss']
        if self.neighborhood_loss_required:
            self.history_users_per_i = {}
            self._get_history_users_i()
            self.user_user_dict = self._get_my_neighbors(self.config['USER_ID_FIELD'])
            self.item_item_dict = self._get_my_neighbors(self.config['ITEM_ID_FIELD'])

    def pretrain_setup(self):
        """
        Reset dataloader. Outputing the same positive & negative samples with each training.
        :return:
        """
        # sort & random
        if self.shuffle:
            self.dataset = self.dataset_bk.copy(self.dataset_bk.df)
        self.all_items.sort()
        if self.use_full_sampling:
            self.all_uids.sort()
        random.shuffle(self.all_items)
        # reorder dataset as default (chronological order)
        #self.dataset.sort_by_chronological()

    def inter_matrix(self, form='coo', value_field=None):
        """Get sparse matrix that describe interactions between user_id and item_id.

        Sparse matrix has shape (user_num, item_num).

        For a row of <src, tgt>, ``matrix[src, tgt] = 1`` if ``value_field`` is ``None``,
        else ``matrix[src, tgt] = self.inter_feat[src, tgt]``.

        Args:
            form (str, optional): Sparse matrix format. Defaults to ``coo``.
            value_field (str, optional): Data of sparse matrix, which should exist in ``df_feat``.
                Defaults to ``None``.

        Returns:
            scipy.sparse: Sparse matrix in form ``coo`` or ``csr``.
        """
        if not self.dataset.uid_field or not self.dataset.iid_field:
            raise ValueError('dataset doesn\'t exist uid/iid, thus can not converted to sparse matrix')
        return self._create_sparse_matrix(self.dataset.df, self.dataset.uid_field,
                                          self.dataset.iid_field, form, value_field)

    def _create_sparse_matrix(self, df_feat, source_field, target_field, form='coo', value_field=None):
        """Get sparse matrix that describe relations between two fields.

        Source and target should be token-like fields.

        Sparse matrix has shape (``self.num(source_field)``, ``self.num(target_field)``).

        For a row of <src, tgt>, ``matrix[src, tgt] = 1`` if ``value_field`` is ``None``,
        else ``matrix[src, tgt] = df_feat[value_field][src, tgt]``.

        Args:
            df_feat (pandas.DataFrame): Feature where src and tgt exist.
            form (str, optional): Sparse matrix format. Defaults to ``coo``.
            value_field (str, optional): Data of sparse matrix, which should exist in ``df_feat``.
                Defaults to ``None``.

        Returns:
            scipy.sparse: Sparse matrix in form ``coo`` or ``csr``.
        """
        src = df_feat[source_field].values
        tgt = df_feat[target_field].values
        if value_field is None:
            data = np.ones(len(df_feat))
        else:
            if value_field not in df_feat.columns:
                raise ValueError('value_field [{}] should be one of `df_feat`\'s features.'.format(value_field))
            data = df_feat[value_field].values
        mat = coo_matrix((data, (src, tgt)), shape=(self.dataset.user_num, self.dataset.item_num))

        if form == 'coo':
            return mat
        elif form == 'csr':
            return mat.tocsr()
        else:
            raise NotImplementedError('sparse matrix format [{}] has not been implemented.'.format(form))

    @property
    def pr_end(self):
        if self.use_full_sampling:
            return len(self.all_uids)
        return len(self.dataset)

    def _shuffle(self):
        self.dataset.shuffle()
        if self.use_full_sampling:
            np.random.shuffle(self.all_uids)

    def _next_batch_data(self):
        return self.sample_func()

    def _get_neg_sample(self):
        cur_data = self.dataset[self.pr: self.pr + self.step]
        self.pr += self.step
        # to tensor
        user_tensor = torch.tensor(cur_data[self.config['USER_ID_FIELD']].values).type(torch.LongTensor).to(self.device)
        item_tensor = torch.tensor(cur_data[self.config['ITEM_ID_FIELD']].values).type(torch.LongTensor).to(self.device)
        batch_tensor = torch.cat((torch.unsqueeze(user_tensor, 0),
                                  torch.unsqueeze(item_tensor, 0)))
        u_ids = cur_data[self.config['USER_ID_FIELD']]
        # sampling negative items only in the dataset (train)
        neg_ids = self._sample_neg_ids(u_ids).to(self.device)
        # for neighborhood loss
        if self.neighborhood_loss_required:
            i_ids = cur_data[self.config['ITEM_ID_FIELD']]
            pos_neighbors, neg_neighbors = self._get_neighborhood_samples(i_ids, self.config['ITEM_ID_FIELD'])
            pos_neighbors, neg_neighbors = pos_neighbors.to(self.device), neg_neighbors.to(self.device)

            batch_tensor = torch.cat((batch_tensor, neg_ids.unsqueeze(0),
                                      pos_neighbors.unsqueeze(0), neg_neighbors.unsqueeze(0)))

        # merge negative samples
        else:
            batch_tensor = torch.cat((batch_tensor, neg_ids.unsqueeze(0)))

        return batch_tensor

In [ ]:
class EvalDataLoader(AbstractDataLoader):
    """
        additional_dataset: training dataset in evaluation
    """
    def __init__(self, config, dataset, additional_dataset=None,
                 batch_size=1, shuffle=False):
        super().__init__(config, dataset, additional_dataset=additional_dataset,
                         batch_size=batch_size, neg_sampling=False, shuffle=shuffle)

        if additional_dataset is None:
            raise ValueError('Training datasets is nan')
        self.eval_items_per_u = []
        self.eval_len_list = []
        self.train_pos_len_list = []

        self.eval_u = self.dataset.df[self.dataset.uid_field].unique()
        # special for eval dataloader
        self.pos_items_per_u = self._get_pos_items_per_u(self.eval_u).to(self.device)
        self._get_eval_items_per_u(self.eval_u)
        # to device
        self.eval_u = torch.tensor(self.eval_u).type(torch.LongTensor).to(self.device)

    @property
    def pr_end(self):
        return self.eval_u.shape[0]

    def _shuffle(self):
        self.dataset.shuffle()

    def _next_batch_data(self):
        inter_cnt = sum(self.train_pos_len_list[self.pr: self.pr+self.step])
        batch_users = self.eval_u[self.pr: self.pr + self.step]
        batch_mask_matrix = self.pos_items_per_u[:, self.inter_pr: self.inter_pr+inter_cnt].clone()
        # user_ids to index
        batch_mask_matrix[0] -= self.pr
        self.inter_pr += inter_cnt
        self.pr += self.step

        return [batch_users, batch_mask_matrix]

    def _get_pos_items_per_u(self, eval_users):
        """
        history items in training dataset.
        masking out positive items in evaluation
        :return:
        user_id - item_ids matrix
        [[0, 0, ... , 1, ...],
         [0, 1, ... , 0, ...]]
        """
        uid_field = self.additional_dataset.uid_field
        iid_field = self.additional_dataset.iid_field
        # load avail items for all uid
        uid_freq = self.additional_dataset.df.groupby(uid_field)[iid_field]
        u_ids = []
        i_ids = []
        for i, u in enumerate(eval_users):
            u_ls = uid_freq.get_group(u).values
            i_len = len(u_ls)
            self.train_pos_len_list.append(i_len)
            u_ids.extend([i]*i_len)
            i_ids.extend(u_ls)
        return torch.tensor([u_ids, i_ids]).type(torch.LongTensor)

    def _get_eval_items_per_u(self, eval_users):
        """
        get evaluated items for each u
        :return:
        """
        uid_field = self.dataset.uid_field
        iid_field = self.dataset.iid_field
        # load avail items for all uid
        uid_freq = self.dataset.df.groupby(uid_field)[iid_field]
        for u in eval_users:
            u_ls = uid_freq.get_group(u).values
            self.eval_len_list.append(len(u_ls))
            self.eval_items_per_u.append(u_ls)
        self.eval_len_list = np.asarray(self.eval_len_list)

    # return pos_items for each u
    def get_eval_items(self):
        return self.eval_items_per_u

    def get_eval_len_list(self):
        return self.eval_len_list

    def get_eval_users(self):
        return self.eval_u.cpu()



4. Run model

In [16]:
############ Dataset loadded, run model
hyper_ret = []
val_metric = config['valid_metric'].lower()
best_test_value = 0.0
idx = best_test_idx = 0

In [18]:
val_metric

'recall@20'

In [19]:
# hyper-parameters
hyper_ls = []
if "seed" not in config['hyper_parameters']:
    config['hyper_parameters'] = ['seed'] + config['hyper_parameters']
for i in config['hyper_parameters']:
    hyper_ls.append(config[i] or [None])

In [21]:
config['hyper_parameters']

['seed', 'n_layers', 'reg_weight', 'dropout']

In [20]:
hyper_ls
# 총 1*2*2*2 = 8 case에 대한 학습을 진행함

[[999], [1, 2], [0.1, 0.01], [0.3, 0.5]]

In [22]:
# combinations
combinators = list(product(*hyper_ls))
total_loops = len(combinators)

In [23]:
# hyperparameter 조정
combinators 

[(999, 1, 0.1, 0.3),
 (999, 1, 0.1, 0.5),
 (999, 1, 0.01, 0.3),
 (999, 1, 0.01, 0.5),
 (999, 2, 0.1, 0.3),
 (999, 2, 0.1, 0.5),
 (999, 2, 0.01, 0.3),
 (999, 2, 0.01, 0.5)]

In [25]:
# 총 학습 횟수
total_loops

8

In [ ]:
for hyper_tuple in combinators:
        # random seed reset
        for j, k in zip(config['hyper_parameters'], hyper_tuple):
            config[j] = k
        init_seed(config['seed'])

        logger.info('========={}/{}: Parameters:{}={}======='.format(
            idx+1, total_loops, config['hyper_parameters'], hyper_tuple))
        # (example)Tue 16 Jul 2024 19:29:13 INFO =========1/8: Parameters:['seed', 'n_layers', 'reg_weight', 'dropout']=(999, 1, 0.1, 0.3)=======

        # set random state of dataloader : TrainDataLoader의 pretrain_setup 함수 - sampling을 위한 과정?
        train_data.pretrain_setup()

        # model loading and initialization
        # get_model : from utils.utils import get_model - return model_class = getattr(model_modeul, model_name)
        model = get_model(config['model'])(config, train_data).to(config['device'])
        logger.info(model)
        """
        (ex)
        Tue 16 Jul 2024 19:29:15 INFO BM3(
    (user_embedding): Embedding(19445, 64)
    (item_id_embedding): Embedding(7050, 64)
    (predictor): Linear(in_features=64, out_features=64, bias=True)
    (reg_loss): EmbLoss()
    (image_embedding): Embedding(7050, 4096)
    (image_trs): Linear(in_features=4096, out_features=64, bias=True)
    (text_embedding): Embedding(7050, 384)
    text_trs): Linear(in_features=384, out_features=64, bias=True)
    )
    Trainable parameters: 33570688  
        """

        # trainer loading and initialization
        trainer = get_trainer()(config, model, mg)
        # MMRec/src/common/trainer.py - class Trainer
        # debug
        # model training
        best_valid_score, best_valid_result, best_test_upon_valid = trainer.fit(train_data, valid_data=valid_data, test_data=test_data, saved=save_model)
        #########
        hyper_ret.append((hyper_tuple, best_valid_result, best_test_upon_valid))

        # save best test
        if best_test_upon_valid[val_metric] > best_test_value:
            best_test_value = best_test_upon_valid[val_metric]
            best_test_idx = idx
        idx += 1

        logger.info('best valid result: {}'.format(dict2str(best_valid_result)))
        logger.info('test result: {}'.format(dict2str(best_test_upon_valid)))
        logger.info('████Current BEST████:\nParameters: {}={},\n'
                    'Valid: {},\nTest: {}\n\n\n'.format(config['hyper_parameters'],
            hyper_ret[best_test_idx][0], dict2str(hyper_ret[best_test_idx][1]), dict2str(hyper_ret[best_test_idx][2])))
        """
        Tue 16 Jul 2024 19:49:15 INFO best valid result: recall@5: 0.0340    recall@10: 0.0538    recall@20: 0.0861    recall@50: 0.1460    ndcg@5: 0.0229    ndcg@10: 0.0293    ndcg@20: 0.0375    ndcg@50: 0.0495    precision@5: 0.0071    precision@10: 0.0057    precision@20: 0.0046    precision@50: 0.0031    map@5: 0.0190    map@10: 0.0216    map@20: 0.0238    map@50: 0.0256    
Tue 16 Jul 2024 19:49:15 INFO test result: recall@5: 0.0331    recall@10: 0.0547    recall@20: 0.0871    recall@50: 0.1493    ndcg@5: 0.0217    ndcg@10: 0.0287    ndcg@20: 0.0371    ndcg@50: 0.0496    precision@5: 0.0074    precision@10: 0.0061    precision@20: 0.0048    precision@50: 0.0033    map@5: 0.0173    map@10: 0.0201    map@20: 0.0224    map@50: 0.0243    
Tue 16 Jul 2024 19:49:15 INFO ████Current BEST████:
Parameters: ['seed', 'n_layers', 'reg_weight', 'dropout']=(999, 1, 0.1, 0.3),
Valid: recall@5: 0.0340    recall@10: 0.0538    recall@20: 0.0861    recall@50: 0.1460    ndcg@5: 0.0229    ndcg@10: 0.0293    ndcg@20: 0.0375    ndcg@50: 0.0495    precision@5: 0.0071    precision@10: 0.0057    precision@20: 0.0046    precision@50: 0.0031    map@5: 0.0190    map@10: 0.0216    map@20: 0.0238    map@50: 0.0256    ,
Test: recall@5: 0.0331    recall@10: 0.0547    recall@20: 0.0871    recall@50: 0.1493    ndcg@5: 0.0217    ndcg@10: 0.0287    ndcg@20: 0.0371    ndcg@50: 0.0496    precision@5: 0.0074    precision@10: 0.0061    precision@20: 0.0048    precision@50: 0.0033    map@5: 0.0173    map@10: 0.0201    map@20: 0.0224    map@50: 0.0243    

        """

In [26]:
get_model(config['model'])

models.bm3.BM3

In [28]:
get_model(config['model'])(config, train_data).to(config['device'])

AttributeError: 'dok_matrix' object has no attribute '_update'

In [ ]:
def pretrain_setup(self):
        """
        Reset dataloader. Outputing the same positive & negative samples with each training.
        :return:
        """
     # sort & random
    if self.shuffle:
        self.dataset = self.dataset_bk.copy(self.dataset_bk.df)
    self.all_items.sort()
    if self.use_full_sampling:
        self.all_uids.sort()
    random.shuffle(self.all_items)
    # reorder dataset as default (chronological order)
    #self.dataset.sort_by_chronological()

In [ ]:
def get_model(model_name):
    r"""Automatically select model class based on model name
    Args:
        model_name (str): model name
    Returns:
        Recommender: model class
    """
    model_file_name = model_name.lower()
    module_path = '.'.join(['models', model_file_name])
    if importlib.util.find_spec(module_path, __name__):
        model_module = importlib.import_module(module_path, __name__)

    model_class = getattr(model_module, model_name)
    return model_class

In [ ]:
def get_trainer():
    return getattr(importlib.import_module('common.trainer'), 'Trainer')
# MMRec/src/common/trainer.py

In [ ]:
class Trainer(AbstractTrainer):
    r"""The basic Trainer for basic training and evaluation strategies in recommender systems. This class defines common
    functions for training and evaluation processes of most recommender system models, including fit(), evaluate(),
   and some other features helpful for model training and evaluation.

   Training과 Evaluation 과정의 common function에 대한 정의 - fit(), evaluate()

    Generally speaking, this class can serve most recommender system models, If the training process of the model is to
    simply optimize a single loss without involving any complex training strategies, such as adversarial learning,
    pre-training and so on.

    Initializing the Trainer needs two parameters: `config` and `model`. `config` records the parameters information
    for controlling training and evaluation, such as `learning_rate`, `epochs`, `eval_step` and so on.
    More information can be found in [placeholder]. `model` is the instantiated object of a Model Class.

    """

    def __init__(self, config, model, mg=False):
        super(Trainer, self).__init__(config, model)

        # 학습 정보 받음
        self.logger = getLogger()
        self.learner = config['learner']
        self.learning_rate = config['learning_rate']
        self.epochs = config['epochs']
        self.eval_step = min(config['eval_step'], self.epochs)
        self.stopping_step = config['stopping_step']
        self.clip_grad_norm = config['clip_grad_norm']
        self.valid_metric = config['valid_metric'].lower()
        self.valid_metric_bigger = config['valid_metric_bigger']
        self.test_batch_size = config['eval_batch_size']
        self.device = config['device']
        self.weight_decay = 0.0
        if config['weight_decay'] is not None:
            wd = config['weight_decay']
            self.weight_decay = eval(wd) if isinstance(wd, str) else wd

        self.req_training = config['req_training']

        self.start_epoch = 0
        self.cur_step = 0

        tmp_dd = {}
        for j, k in list(itertools.product(config['metrics'], config['topk'])):
            tmp_dd[f'{j.lower()}@{k}'] = 0.0
        self.best_valid_score = -1
        self.best_valid_result = tmp_dd
        self.best_test_upon_valid = tmp_dd
        self.train_loss_dict = dict()
        self.optimizer = self._build_optimizer()

        #fac = lambda epoch: 0.96 ** (epoch / 50)
        lr_scheduler = config['learning_rate_scheduler']        # check zero?
        fac = lambda epoch: lr_scheduler[0] ** (epoch / lr_scheduler[1])
        scheduler = optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda=fac)
        self.lr_scheduler = scheduler

        self.eval_type = config['eval_type']
        self.evaluator = TopKEvaluator(config)

        self.item_tensor = None
        self.tot_item_num = None
        self.mg = mg
        self.alpha1 = config['alpha1']
        self.alpha2 = config['alpha2']
        self.beta = config['beta']


In [ ]:
    def fit(self, train_data, valid_data=None, test_data=None, saved=False, verbose=True):
        r"""Train the model based on the train data and the valid data.
        train data와 valid data에 기초해서 모델을 학습 시킴/

        Args:
            train_data (DataLoader): the train data
            valid_data (DataLoader, optional): the valid data, default: None.
                                               If it's None, the early_stopping is invalid.
            test_data (DataLoader, optional): None
            verbose (bool, optional): whether to write training and evaluation information to logger, default: True
            saved (bool, optional): whether to save the model parameters, default: True

        Returns:
             (float, dict): best valid score and best valid result. If valid_data is None, it returns (-1, None)
        """
        for epoch_idx in range(self.start_epoch, self.epochs):
            # train
            training_start_time = time()
            self.model.pre_epoch_processing()
            train_loss, _ = self._train_epoch(train_data, epoch_idx) # train_epoch  함수에서 loss
            if torch.is_tensor(train_loss):
                # get nan loss
                break
            #for param_group in self.optimizer.param_groups:
            #    print('======lr: ', param_group['lr'])
            self.lr_scheduler.step()

            # train loss dictionary에 저장
            self.train_loss_dict[epoch_idx] = sum(train_loss) if isinstance(train_loss, tuple) else train_loss
            training_end_time = time()
            train_loss_output = \
                self._generate_train_loss_output(epoch_idx, training_start_time, training_end_time, train_loss)
            post_info = self.model.post_epoch_processing()
            if verbose:
                self.logger.info(train_loss_output)
                if post_info is not None:
                    self.logger.info(post_info)

            # eval: To ensure the test result is the best model under validation data, set self.eval_step == 1
            if (epoch_idx + 1) % self.eval_step == 0:
                valid_start_time = time()
                valid_score, valid_result = self._valid_epoch(valid_data)
                self.best_valid_score, self.cur_step, stop_flag, update_flag = early_stopping(
                    valid_score, self.best_valid_score, self.cur_step,
                    max_step=self.stopping_step, bigger=self.valid_metric_bigger)
                valid_end_time = time()
                valid_score_output = "epoch %d evaluating [time: %.2fs, valid_score: %f]" % \
                                     (epoch_idx, valid_end_time - valid_start_time, valid_score)
                valid_result_output = 'valid result: \n' + dict2str(valid_result)
                # test
                _, test_result = self._valid_epoch(test_data)
                if verbose:
                    self.logger.info(valid_score_output)
                    self.logger.info(valid_result_output)
                    self.logger.info('test result: \n' + dict2str(test_result))
                if update_flag:
                    update_output = '██ ' + self.config['model'] + '--Best validation results updated!!!'
                    if verbose:
                        self.logger.info(update_output)
                    self.best_valid_result = valid_result
                    self.best_test_upon_valid = test_result

                if stop_flag:
                    stop_output = '+++++Finished training, best eval result in epoch %d' % \
                                  (epoch_idx - self.cur_step * self.eval_step)
                    if verbose:
                        self.logger.info(stop_output)
                    break
        return self.best_valid_score, self.best_valid_result, self.best_test_upon_valid


In [ ]:
# log info
    logger.info('\n============All Over=====================')
    for (p, k, v) in hyper_ret:
        logger.info('Parameters: {}={},\n best valid: {},\n best test: {}'.format(config['hyper_parameters'],
                                                                                  p, dict2str(k), dict2str(v)))

    logger.info('\n\n█████████████ BEST ████████████████')
    logger.info('\tParameters: {}={},\nValid: {},\nTest: {}\n\n'.format(config['hyper_parameters'],
                                                                   hyper_ret[best_test_idx][0],
                                                                   dict2str(hyper_ret[best_test_idx][1]),
                                                                   dict2str(hyper_ret[best_test_idx][2])))


In [33]:
import itertools
list(itertools.product(config['metrics'], config['topk']))

[('Recall', 5),
 ('Recall', 10),
 ('Recall', 20),
 ('Recall', 50),
 ('NDCG', 5),
 ('NDCG', 10),
 ('NDCG', 20),
 ('NDCG', 50),
 ('Precision', 5),
 ('Precision', 10),
 ('Precision', 20),
 ('Precision', 50),
 ('MAP', 5),
 ('MAP', 10),
 ('MAP', 20),
 ('MAP', 50)]